In [183]:
import datetime
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
#import xvec
import shapely
from shapely import Point, box
from rasterio.features import rasterize
import rasterio


In [184]:
aoi = gpd.read_file('/mnt/CEPH_PROJECTS/ScaleAgData/03_Ancillary_Data/LAFIS_zonal_stats_prato_stabile_1000.shp')
grid5 = pd.read_csv('/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/polygons_grid_5')
s1 = xr.open_dataset(f'/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/latest/GRD_luca/asc/grid5/S1_grid5_1/openEO.nc')
#aoi = aoi.to_crs(4326)

In [185]:
s1

<xarray.Dataset>
Dimensions:  (t: 12, x: 3764, y: 2282)
Coordinates:
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
Data variables:
    crs      |S1 ...
    VV       (t, y, x) float32 ...
    VH       (t, y, x) float32 ...
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [186]:
if 'crs' in s1:
    crs_info = s1['crs'].attrs  # Get CRS info as a string
    s1.attrs['crs'] = crs_info  # Move CRS info to 'attribute'
    s1 = s1.drop_vars('crs') 

In [187]:
s1

<xarray.Dataset>
Dimensions:  (t: 12, x: 3764, y: 2282)
Coordinates:
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
Data variables:
    VV       (t, y, x) float32 ...
    VH       (t, y, x) float32 ...
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:        
    crs:          {'crs_wkt': 'PROJCS["WGS 84 / UTM zone 32N", GEOGCS["WGS 84...

In [188]:
grid5['geometry'] = grid5['geometry'].apply(lambda x: shapely.wkt.loads(x))
gdf_grid5 = gpd.GeoDataFrame(grid5, geometry='geometry')
gdf_grid5.crs = 'EPSG:4326'
gdf_grid5 = gdf_grid5.to_crs(32632)
gdf_grid5.head(3)

,Unnamed: 0,ID,FID,CUAA,CODE,DESCR_IT,DESCR_DE,area,dem_mean,dem_median,slo_mean,slo_median,asp_mean,asp_median,north_mean,north_medi,dem_class,geometry
0,0,2894152,PRGMFR75E13F132Z-1688030,PRGMFR75E13F132Z,AP2,Prato stabile,Wiese (Dauerwiese),56,1301.867221,1301.705200,17.897289,17.806307,85.131834,84.665092,1.000000,1.0,1300.0,"POLYGON ((655594.691 5148086.447, 655600.714 5..."
1,1,2347633,RNRGTH80H18F132V-59,RNRGTH80H18F132V,AP2,Prato stabile,Wiese (Dauerwiese),31147,1302.440111,1302.553589,22.939459,22.398680,145.304115,153.931000,0.043887,0.0,1300.0,"POLYGON ((648855.470 5168788.793, 648856.965 5..."
2,2,2970168,WSSCRS85L19A952F-2158250,WSSCRS85L19A952F,AP2,Prato stabile,Wiese (Dauerwiese),842,1318.083754,1317.940063,12.269650,12.262682,228.961556,228.782425,0.000000,0.0,1300.0,"POLYGON ((663622.297 5151188.843, 663645.031 5..."


In [189]:
# Step 1: Convert xarray to georeferenced raster
latitude = s1.coords['y']
longitude = s1.coords['x']

transform = rasterio.transform.from_origin(longitude.min(), latitude.max(), longitude[1]-longitude[0], latitude[0]-latitude[1])


In [212]:
# Step 2: Rasterize the shapefile polygons
shapefile_rasterized = rasterize(
    #[(geometry, 1) for geometry in gdf_grid5.geometry],
    [(geometry, value) for geometry, value in zip(gdf_grid5.geometry, gdf_grid5['ID'])],
    out_shape=(s1.sizes['y'], s1.sizes['x']),
    transform=transform,
    #dtype=np.uint8,
    dtype=np.float32,
    all_touched=True)

In [213]:
shapefile_rasterized

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [219]:
np.unique(shapefile_rasterized)

array([      0.,  124971.,  336370.,  336766.,  336774.,  374933.,
        448856.,  789571.,  895508.,  960894.,  966658., 1030731.,
       1055730., 1241504., 1314317., 1517653., 1877555., 1952504.,
       1957080., 1958022., 1958073., 1961590., 1963914., 1970639.,
       1970645., 1972334., 1990763., 2016048., 2019740., 2031652.,
       2058533., 2060657., 2084570., 2099309., 2121508., 2137896.,
       2137965., 2174311., 2199380., 2200471., 2304373., 2347507.,
       2347633., 2380986., 2457286., 2467073., 2534765., 2548713.,
       2570033., 2570048., 2580937., 2582229., 2609334., 2614883.,
       2635948., 2687416., 2687420., 2702272., 2703270., 2705726.,
       2713883., 2714259., 2736238., 2737615., 2745823., 2754446.,
       2765541., 2767485., 2782118., 2818758., 2836866., 2859896.,
       2864690., 2871558., 2876342., 2879755., 2883063., 2884034.,
       2887917., 2887921., 2888581., 2894152., 2902867., 2915328.,
       2948894., 2958313., 2960421., 2961840., 2970168., 29753

In [205]:
# Convert rasterized array to xarray DataArray with latitude and longitude coordinates
rasterized_da = xr.DataArray(shapefile_rasterized, coords={'y': latitude, 'x': longitude}, dims=['y', 'x'])

In [206]:
rasterized_da

<xarray.DataArray (y: 2282, x: 3764)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05

In [207]:
# Multiply the xarray data cube with the rasterized array
aggregated_data = s1 * rasterized_da

In [208]:
aggregated_data

<xarray.Dataset>
Dimensions:  (t: 12, x: 3764, y: 2282)
Coordinates:
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
Data variables:
    VV       (t, y, x) float32 nan nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    VH       (t, y, x) float32 nan nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0

In [209]:
# Compute the aggregated result, for example taking mean over time
aggregated_result = aggregated_data.mean(dim='t')

In [210]:
aggregated_result

<xarray.Dataset>
Dimensions:  (x: 3764, y: 2282)
Coordinates:
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
Data variables:
    VV       (y, x) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    VH       (y, x) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0

In [211]:
np.unique(aggregated_result.VV.values)

array([0.0000000e+00, 4.9270088e+03, 5.9309399e+03, ..., 3.8962388e+07,
                 inf,           nan], dtype=float32)

In [223]:
flattened_array = shapefile_rasterized.flatten()
flattened_array.shape

(8589448,)

In [225]:
new_aggregated_result = aggregated_result.assign_coords(index=flattened_array)


In [226]:
new_aggregated_result

<xarray.Dataset>
Dimensions:  (x: 3764, y: 2282, index: 8589448)
Coordinates:
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
  * index    (index) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
Data variables:
    VV       (y, x) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    VH       (y, x) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0

In [236]:
# how to assign id to pixels according to the polygon
c = new_aggregated_result.groupby('index').mean('index')

In [237]:
c

<xarray.Dataset>
Dimensions:  (x: 3764, y: 2282, index: 102)
Coordinates:
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
  * index    (index) float32 0.0 1.25e+05 3.364e+05 ... 3.049e+06 3.107e+06
Data variables:
    VV       (index, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    VH       (index, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0